# Introduction to Quantum Safe Cryptography

This notebook serves as a compilation of the hands-on examples provided in the 'Introduction to Quantum Safe Cryptography' provided by IBM.

Requirements:

-   'cryptography' library 

## CHFs: Cryptographic HASH FUNCTIONS

### The SHA-256 algorithm

Let us demonstrate cryptographic hashing my making use of the well-known SHA-256 algorithm, widely used in modern technologies such as the Blockchain (cryptocurrencies), digital signatures and certificates.

A good hash function will produce digests with large differences even for input strings with small variations.

In [1]:
# Begin by importing the necessary modules

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes

We generate now two very similar messages which we will feed to the hash function to compare the outputs. 

In [2]:
#Helper function that returns the number of characters different in two strings

def char_diff(str1, str2):
    return sum ( str1[i] != str2[i] for i in range(len(str1)) )

# Messages to be hashed

message_1 = b"Buy 10000 shares of WXYZ stock now!"
message_2 = b"Buy 10000 shares of VXYZ stock now!"

print(f"The two messages differ by { char_diff(message_1, message_2)} characters")

The two messages differ by 1 characters


We now have to instantiate a hash object and update it with the message to be hashed. We then finalize the hash and print the digest. This involves the calls to three methods.

In [3]:
# Create new SHA-256 hash objects, one for each message

chf_1 = hashes.Hash(hashes.SHA256(), backend=default_backend())
chf_2 = hashes.Hash(hashes.SHA256(), backend=default_backend())

# Update each hash object with the bytes of the corresponding message

chf_1.update(message_1)
chf_2.update(message_2)

# Finalize the hash process and obtain the digests

digest_1 = chf_1.finalize()
digest_2 = chf_2.finalize()

In [4]:
digest_1

b'n\x0eba\xb7\x13\x1b\xd8\x0f\xfd\xb2\xa4\xd4/\x9d\x04&65\x0eE\xe1\x84\xb9/\xcb\xcc\x96F\xea\xf1\xe7'

We need to convert the digests to some more readable (or printeable) format, e.g. hexadecimal strings.

In [5]:
#Convert the resulting hash to hexadecimal strings for convenient printing

digest_1_str = digest_1.hex()
digest_2_str = digest_2.hex()

#Print out the digests as strings 

print(f"digest-1: {digest_1_str}")
print(f"digest-2: {digest_2_str}")

print(f"The two digests differ by { char_diff(digest_1_str, digest_2_str)} characters")

digest-1: 6e0e6261b7131bd80ffdb2a4d42f9d042636350e45e184b92fcbcc9646eaf1e7
digest-2: 6b0abb368c3a1730f935b68105e3f3ae7fd43d7e786d3ed3503dbb45c74ada46
The two digests differ by 57 characters


This shows the 'avalanche effect' in the SHA-256 algorithm. We started with two strings difering only in one character. The digests then differ by 57 characters!